# Chains in LangChain

Watch:
https://learn.deeplearning.ai/langchain/lesson/4/chains 

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [6]:
llm_model: str = "gpt-3.5-turbo-0613"

In [2]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [3]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain: LLMChain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product: str = "Queen Size Sheet Set"
response: str = chain.run(product)

'"RoyalRest Bedding" would be a suitable name to describe a company that makes Queen Size Sheet Sets.'

## SimpleSequentialChain

One input and one output

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one: LLMChain = LLMChain(llm=llm, prompt=first_prompt)

In [14]:
# prompt template 2
second_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two : LLMChain = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain: SimpleSequentialChain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose = True)

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens offers luxurious and elegant bedding and home décor products, adding a touch of opulence to any space.

> Finished chain.


'Regal Linens offers luxurious and elegant bedding and home décor products, adding a touch of opulence to any space.'

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm: ChatOpenAI = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one: LLMChain = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [21]:
second_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two: LLMChain = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three: LLMChain = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [23]:
# prompt template 4: follow up message
fourth_prompt: ChatPromptTemplate = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four: LLMChain = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain: SequentialChain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste of the product, mentioning that the foam is odd and believes that it might be an old batch or counterfeit.',
 'followup_message': "Réponse de suivi: Nous sommes désolés d'apprendre que vous n'avez pas été satisfait(e) du goût de notre produit. Votre avis est précieux et nous tenons à vous assurer que nous prenons vos préoccupations très au sérieux. Nous nous efforçons constamment d'offrir une expérience gustative exceptionnelle à nos clients. \n\nConcernant la mousse étrange que vous avez mentionnée, nous tenons à préciser que notre produit est fabriqué en utilisant uniquement des ing

## Router Chain